In [1]:
import numpy as np
import xarray as xr
import pandas as pd
import shutil
import h5py
import os,json
from pprint import pprint

In [2]:
#change working directory
%cd ../

/home/jovyan/icepyx


In [11]:
%load_ext autoreload
%autoreload 2

from icepyx import is2class as ipd

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Choose a region for subsetting as well. Use the same region as in the core demo.

In [12]:
region_a = ipd.Icesat2Data('ATL09',[-180, 60, 180, 90],['2018-10-18','2018-10-18'], \
                           start_time='00:00:00', end_time='23:59:59', version='2')

In [13]:
region_07 = ipd.Icesat2Data('ATL07',[-55, 68, -48, 71],['2019-02-22','2019-02-28'], \
                           start_time='00:00:00', end_time='23:59:59', version='2')

In [14]:
session=region_a.earthdata_login('liuzheng','liuzheng@apl.uw.edu')

Earthdata Login password:  ········


### Now, generate variable dictionary. 
Get the variable dictionary by parsing the dataset xml information from NSIDC, by calling ```show_custom_options(session)```. 

The data variables are stored in ```region_a._cust_options['variables']```. 

In [7]:
opts = region_a.show_custom_options(session,dictview=True)

Subsetting options
[{'id': 'ICESAT2',
  'maxGransAsyncRequest': '2000',
  'maxGransSyncRequest': '100',
  'spatialSubsetting': 'true',
  'spatialSubsettingShapefile': 'true',
  'temporalSubsetting': 'true',
  'type': 'both'}]
Data File Formats (Reformatting Options)
['TABULAR_ASCII', 'NetCDF4-CF', 'Shapefile', 'NetCDF-3']
Reprojection Options
[]
Data File (Reformatting) Options Supporting Reprojection
['TABULAR_ASCII', 'NetCDF4-CF', 'Shapefile', 'NetCDF-3', 'No reformatting']
Data File (Reformatting) Options NOT Supporting Reprojection
[]
Data Variables (also Subsettable)
{'a_m1': ['ancillary_data/atmosphere/a_m1'],
 'a_m2': ['ancillary_data/atmosphere/a_m2'],
 'aclr_true': ['profile_1/high_rate/aclr_true',
               'profile_2/high_rate/aclr_true',
               'profile_3/high_rate/aclr_true'],
 'aclr_use_atlas': ['ancillary_data/atmosphere/aclr_use_atlas'],
 'alpha': ['ancillary_data/atmosphere/alpha'],
 'apparent_surf_reflec': ['profile_1/high_rate/apparent_surf_reflec',
    

#### TESTING SECTION:
##### Setup the user provided variable list to subset variables: Please TRY OUT the tests below. 

Options for inputting variables:
1. Use a default list for the dataset (not yet fully implemented across all datasets)
2. Provide a list of variable names, which will return all path-variable combinations (e.g. longitude will return longitude for both beams for all profiles)
3. Provide a list of variable names and/or specific profiles/beams (not yet implemented).

An example of each type of input is below.

#### Test 1:
Add ```latitude``` for profile 1 and 2

In [15]:
#default variables
region_a.build_wanted_var_list(beam_list=['profile_2'],clear=True)
region_a.variables

{'sc_orient': ['orbit_info/sc_orient'],
 'atlas_sdp_gps_epoch': ['ancillary_data/atlas_sdp_gps_epoch'],
 'data_start_utc': ['ancillary_data/data_start_utc'],
 'data_end_utc': ['ancillary_data/data_end_utc'],
 'granule_start_utc': ['ancillary_data/granule_start_utc'],
 'granule_end_utc': ['ancillary_data/granule_end_utc'],
 'start_delta_time': ['ancillary_data/start_delta_time'],
 'end_delta_time': ['ancillary_data/end_delta_time'],
 'delta_time': ['profile_2/bckgrd_atlas/delta_time',
  'profile_2/high_rate/delta_time',
  'profile_2/low_rate/delta_time',
  'quality_assessment/profile_2/delta_time'],
 'latitude': ['profile_2/high_rate/latitude', 'profile_2/low_rate/latitude'],
 'longitude': ['profile_2/high_rate/longitude',
  'profile_2/low_rate/longitude'],
 'bsnow_h': ['profile_2/high_rate/bsnow_h', 'profile_2/low_rate/bsnow_h'],
 'bsnow_dens': ['profile_2/high_rate/bsnow_dens'],
 'bsnow_con': ['profile_2/high_rate/bsnow_con',
  'profile_2/low_rate/bsnow_con'],
 'bsnow_psc': ['profile_

In [16]:
region_a.build_wanted_var_list(beam_list=['profile_2'],keyword_list=['high_rate'],append=False)
region_a.variables

{'sc_orient': ['orbit_info/sc_orient'],
 'atlas_sdp_gps_epoch': ['ancillary_data/atlas_sdp_gps_epoch'],
 'data_start_utc': ['ancillary_data/data_start_utc'],
 'data_end_utc': ['ancillary_data/data_end_utc'],
 'granule_start_utc': ['ancillary_data/granule_start_utc'],
 'granule_end_utc': ['ancillary_data/granule_end_utc'],
 'start_delta_time': ['ancillary_data/start_delta_time'],
 'end_delta_time': ['ancillary_data/end_delta_time'],
 'delta_time': ['profile_2/high_rate/delta_time'],
 'latitude': ['profile_2/high_rate/latitude'],
 'longitude': ['profile_2/high_rate/longitude'],
 'bsnow_h': ['profile_2/high_rate/bsnow_h'],
 'bsnow_dens': ['profile_2/high_rate/bsnow_dens'],
 'bsnow_con': ['profile_2/high_rate/bsnow_con'],
 'bsnow_psc': ['profile_2/high_rate/bsnow_psc'],
 'bsnow_od': ['profile_2/high_rate/bsnow_od'],
 'cloud_flag_asr': ['profile_2/high_rate/cloud_flag_asr'],
 'cloud_fold_flag': ['profile_2/high_rate/cloud_fold_flag'],
 'cloud_flag_atm': ['profile_2/high_rate/cloud_flag_atm'

In [17]:
var_list = ['orbit_number','cycle_number','rgt'] # add sc_orient_time for transition granules/day
region_a.build_wanted_var_list(var_list=var_list,defaults=False)
region_a.variables

{'sc_orient': ['orbit_info/sc_orient'],
 'atlas_sdp_gps_epoch': ['ancillary_data/atlas_sdp_gps_epoch'],
 'data_start_utc': ['ancillary_data/data_start_utc'],
 'data_end_utc': ['ancillary_data/data_end_utc'],
 'granule_start_utc': ['ancillary_data/granule_start_utc'],
 'granule_end_utc': ['ancillary_data/granule_end_utc'],
 'start_delta_time': ['ancillary_data/start_delta_time'],
 'end_delta_time': ['ancillary_data/end_delta_time'],
 'delta_time': ['profile_2/high_rate/delta_time'],
 'latitude': ['profile_2/high_rate/latitude'],
 'longitude': ['profile_2/high_rate/longitude'],
 'bsnow_h': ['profile_2/high_rate/bsnow_h'],
 'bsnow_dens': ['profile_2/high_rate/bsnow_dens'],
 'bsnow_con': ['profile_2/high_rate/bsnow_con'],
 'bsnow_psc': ['profile_2/high_rate/bsnow_psc'],
 'bsnow_od': ['profile_2/high_rate/bsnow_od'],
 'cloud_flag_asr': ['profile_2/high_rate/cloud_flag_asr'],
 'cloud_fold_flag': ['profile_2/high_rate/cloud_fold_flag'],
 'cloud_flag_atm': ['profile_2/high_rate/cloud_flag_atm'

In [18]:
region_a.build_CMR_params()
region_a.build_reqconfig_params('download')

In [19]:
region_a.build_subset_params(**{'Coverage':region_a.variables})
region_a.subsetparams

{'time': '2018-10-18T00:00:00,2018-10-18T23:59:59',
 'bbox': '-180,60,180,90',
 'Coverage': '/orbit_info/sc_orient,/ancillary_data/atlas_sdp_gps_epoch,/ancillary_data/data_start_utc,/ancillary_data/data_end_utc,/ancillary_data/granule_start_utc,/ancillary_data/granule_end_utc,/ancillary_data/start_delta_time,/ancillary_data/end_delta_time,/profile_2/high_rate/delta_time,/profile_2/high_rate/latitude,/profile_2/high_rate/longitude,/profile_2/high_rate/bsnow_h,/profile_2/high_rate/bsnow_dens,/profile_2/high_rate/bsnow_con,/profile_2/high_rate/bsnow_psc,/profile_2/high_rate/bsnow_od,/profile_2/high_rate/cloud_flag_asr,/profile_2/high_rate/cloud_fold_flag,/profile_2/high_rate/cloud_flag_atm,/profile_2/high_rate/column_od_asr,/profile_2/high_rate/column_od_asr_qf,/profile_2/high_rate/layer_attr,/profile_2/high_rate/layer_bot,/profile_2/high_rate/layer_top,/profile_2/high_rate/layer_flag,/profile_2/high_rate/layer_dens,/profile_2/high_rate/layer_ib,/profile_2/high_rate/msw_flag,/profile_2/hi

In [21]:
region_a.order_granules(verbose=True)

{'feed': {'updated': '2020-04-15T00:42:26.160Z', 'id': 'https://cmr.earthdata.nasa.gov:443/search/granules.json?short_name=ATL09&version=002&temporal=2018-10-18T00%3A00%3A00Z%2C2018-10-18T23%3A59%3A59Z&bounding_box=-180%2C60%2C180%2C90&page_size=10&page_num=1', 'title': 'ECHO granule metadata', 'entry': [{'producer_granule_id': 'ATL09_20181017235527_02960101_002_01.h5', 'time_start': '2018-10-17T23:55:26.000Z', 'orbit': {'ascending_crossing': '-127.82682215638665', 'start_lat': '0', 'start_direction': 'A', 'end_lat': '0', 'end_direction': 'A'}, 'updated': '2019-12-15T08:32:26.279Z', 'orbit_calculated_spatial_domains': [{'equator_crossing_date_time': '2018-10-17T23:55:26.339Z', 'equator_crossing_longitude': '-127.82682215638665', 'orbit_number': '497'}], 'dataset_id': 'ATLAS/ICESat-2 L3A Calibrated Backscatter Profiles and Atmospheric Layer Characteristics V002', 'data_center': 'NSIDC_ECS', 'title': 'SC:ATL09.002:166040943', 'coordinate_system': 'ORBIT', 'time_end': '2018-10-18T01:29:44

In [34]:
oidfn = 'orderID.json'
with open(oidfn,'w') as fid:
    json.dump({'orderID': region_a.orderIDs},fid)

In [35]:
with open(oidfn,'r') as fid:
    ndict = json.load(fid)

In [30]:
region_a.download_granules(session,'down')

Beginning download of zipped output...
Data request 5000000561892 of  2  order(s) is complete.
Beginning download of zipped output...
Data request 5000000561905 of  2  order(s) is complete.


In [32]:

old_dir = 'down'
new_dir = 'ATL09/profile_2/'
for root, dirs, files in os.walk(old_dir, topdown=False):
    for file in files:
        try:
            shutil.move(os.path.join(root, file), new_dir)
        except OSError:
            pass
        
for root, dirs, files in os.walk(old_dir):
    for name in dirs:
        os.rmdir(os.path.join(root, name))

In [8]:
allfiles

['down/166041204/processed_ATL09_20181018092110_03020101_002_01.h5',
 'down/166041288/processed_ATL09_20181018122945_03040101_002_01.h5',
 'down/166040841/processed_ATL09_20181018061236_03000101_002_01.h5',
 'down/166040772/processed_ATL09_20181018074653_03010101_002_01.h5',
 'down/166040943/processed_ATL09_20181017235527_02960101_002_01.h5',
 'down/166041397/processed_ATL09_20181018105528_03030101_002_01.h5',
 'down/166041362/processed_ATL09_20181018153820_03060101_002_01.h5',
 'down/166041380/processed_ATL09_20181018140403_03050101_002_01.h5',
 'down/166040819/processed_ATL09_20181018012943_02970101_002_01.h5']

#### Test 2:
Add ```latitude``` for profile 2 and overwrite

In [39]:
region_a.build_wanted_var_list(beam_list=['profile_2'],var_list=['latitude'], append=False)
region_a.variables

{'sc_orient': ['orbit_info/sc_orient'],
 'atlas_sdp_gps_epoch': ['ancillary_data/atlas_sdp_gps_epoch'],
 'data_start_utc': ['ancillary_data/data_start_utc'],
 'data_end_utc': ['ancillary_data/data_end_utc'],
 'granule_start_utc': ['ancillary_data/granule_start_utc'],
 'granule_end_utc': ['ancillary_data/granule_end_utc'],
 'start_delta_time': ['ancillary_data/start_delta_time'],
 'end_delta_time': ['ancillary_data/end_delta_time'],
 'latitude': ['profile_2/high_rate/latitude', 'profile_2/low_rate/latitude']}

#### Test 2B:
Add ```latitude``` for profile 3 and overwrite (so profile_2 should be removed)

In [40]:
region_a.build_wanted_var_list(beam_list=['profile_3'],var_list=['latitude'],append=False)
region_a.variables

{'sc_orient': ['orbit_info/sc_orient'],
 'atlas_sdp_gps_epoch': ['ancillary_data/atlas_sdp_gps_epoch'],
 'data_start_utc': ['ancillary_data/data_start_utc'],
 'data_end_utc': ['ancillary_data/data_end_utc'],
 'granule_start_utc': ['ancillary_data/granule_start_utc'],
 'granule_end_utc': ['ancillary_data/granule_end_utc'],
 'start_delta_time': ['ancillary_data/start_delta_time'],
 'end_delta_time': ['ancillary_data/end_delta_time'],
 'latitude': ['profile_3/high_rate/latitude', 'profile_3/low_rate/latitude']}

#### Test 3:
Add ```latitude``` for all profiles and with keyword ```low_rate``` and append

In [31]:
#region_a.build_wanted_var_list(beam_list=['profile_3'],var_list=['latitude'],keyword_list=['low_rate'],append=True)
region_a.build_wanted_var_list(var_list=['latitude'],keyword_list=['low_rate'],append=True)
region_a.variables

{'sc_orient': ['orbit_info/sc_orient'],
 'atlas_sdp_gps_epoch': ['ancillary_data/atlas_sdp_gps_epoch'],
 'data_start_utc': ['ancillary_data/data_start_utc'],
 'data_end_utc': ['ancillary_data/data_end_utc'],
 'granule_start_utc': ['ancillary_data/granule_start_utc'],
 'granule_end_utc': ['ancillary_data/granule_end_utc'],
 'start_delta_time': ['ancillary_data/start_delta_time'],
 'end_delta_time': ['ancillary_data/end_delta_time'],
 'latitude': ['profile_2/high_rate/latitude',
  'profile_2/low_rate/latitude',
  'profile_1/low_rate/latitude',
  'profile_3/low_rate/latitude']}

#### Before Test 4:
Go back to test 2. Overwrite ```latitude``` for profile 2 only.

In [32]:
region_a.build_wanted_var_list(beam_list=['profile_2'],var_list=['latitude'],append=False)
region_a.variables

{'sc_orient': ['orbit_info/sc_orient'],
 'atlas_sdp_gps_epoch': ['ancillary_data/atlas_sdp_gps_epoch'],
 'data_start_utc': ['ancillary_data/data_start_utc'],
 'data_end_utc': ['ancillary_data/data_end_utc'],
 'granule_start_utc': ['ancillary_data/granule_start_utc'],
 'granule_end_utc': ['ancillary_data/granule_end_utc'],
 'start_delta_time': ['ancillary_data/start_delta_time'],
 'end_delta_time': ['ancillary_data/end_delta_time'],
 'latitude': ['profile_2/high_rate/latitude', 'profile_2/low_rate/latitude']}

#### Test 5:
Append ```latitude``` for profile 3 and ```high_rate``` only

In [33]:
region_a.build_wanted_var_list(beam_list=['profile_3'],var_list=['latitude'],keyword_list=['low_rate'])
region_a.variables

{'sc_orient': ['orbit_info/sc_orient'],
 'atlas_sdp_gps_epoch': ['ancillary_data/atlas_sdp_gps_epoch'],
 'data_start_utc': ['ancillary_data/data_start_utc'],
 'data_end_utc': ['ancillary_data/data_end_utc'],
 'granule_start_utc': ['ancillary_data/granule_start_utc'],
 'granule_end_utc': ['ancillary_data/granule_end_utc'],
 'start_delta_time': ['ancillary_data/start_delta_time'],
 'end_delta_time': ['ancillary_data/end_delta_time'],
 'latitude': ['profile_2/high_rate/latitude',
  'profile_2/low_rate/latitude',
  'profile_1/low_rate/latitude',
  'profile_3/high_rate/latitude',
  'profile_3/low_rate/latitude']}

#### Test 6:
Add ```sc_orient_time``` under ```orbit_info```.

In [34]:
region_a.build_wanted_var_list(keyword_list=['orbit_info'],var_list=['sc_orient_time'])
region_a.variables

{'sc_orient': ['orbit_info/sc_orient'],
 'atlas_sdp_gps_epoch': ['ancillary_data/atlas_sdp_gps_epoch'],
 'data_start_utc': ['ancillary_data/data_start_utc'],
 'data_end_utc': ['ancillary_data/data_end_utc'],
 'granule_start_utc': ['ancillary_data/granule_start_utc'],
 'granule_end_utc': ['ancillary_data/granule_end_utc'],
 'start_delta_time': ['ancillary_data/start_delta_time'],
 'end_delta_time': ['ancillary_data/end_delta_time'],
 'latitude': ['profile_2/high_rate/latitude',
  'profile_2/low_rate/latitude',
  'profile_1/low_rate/latitude',
  'profile_3/high_rate/latitude',
  'profile_3/low_rate/latitude'],
 'sc_orient_time': ['orbit_info/sc_orient_time']}

#### Test 7:
Add all variables under ```orbit_info``` but path to ```sc_orient_time``` should not be duplicated.

In [35]:
region_a.build_wanted_var_list(keyword_list=['orbit_info'],append=True)
region_a.variables

{'sc_orient': ['orbit_info/sc_orient'],
 'atlas_sdp_gps_epoch': ['ancillary_data/atlas_sdp_gps_epoch'],
 'data_start_utc': ['ancillary_data/data_start_utc'],
 'data_end_utc': ['ancillary_data/data_end_utc'],
 'granule_start_utc': ['ancillary_data/granule_start_utc'],
 'granule_end_utc': ['ancillary_data/granule_end_utc'],
 'start_delta_time': ['ancillary_data/start_delta_time'],
 'end_delta_time': ['ancillary_data/end_delta_time'],
 'latitude': ['profile_2/high_rate/latitude',
  'profile_2/low_rate/latitude',
  'profile_1/low_rate/latitude',
  'profile_3/high_rate/latitude',
  'profile_3/low_rate/latitude'],
 'sc_orient_time': ['orbit_info/sc_orient_time'],
 'crossing_time': ['orbit_info/crossing_time'],
 'cycle_number': ['orbit_info/cycle_number'],
 'lan': ['orbit_info/lan'],
 'orbit_number': ['orbit_info/orbit_number'],
 'rgt': ['orbit_info/rgt']}

#### Test 8:
Add all defaults for all beams and all keywords. After this, have to reinitialize ```region_a``` and regenerate variable dictionary to run the above tests again (unless you set ```append=False```). 

In [36]:
region_a.build_wanted_var_list(defaults=True)

In [ ]:
#variable names + beams/profiles
###STILL NEED TO MAKE THE BELOW POSSIBLE IN THE CODE

### Setting params and download

In [ ]:
region_a.build_CMR_params()
region_a.build_reqconfig_params('download')

In [ ]:
region_a.build_subset_params(**{'Coverage':var_dict})
region_a.subsetparams

In [ ]:
#Identical to above block, but enters the keywords with a different style
region_a.build_subset_params(Coverage=var_dict)
region_a.subsetparams

In [ ]:
region_a.order_granules(session, verbose=True)

In [ ]:
region_a.download_granules(session,'.')

### Examine downloaded subset data file 


In [ ]:
fn = '166458094/processed_ATL09_20190222003738_08490201_002_01.h5'

#### Check the downloaded dataset
Take ```latitude``` for example,

In [ ]:
varname = 'latitude'
#varname = 'sc_orient'

varlist = []
def IS2h5walk(vname, h5node):
    if isinstance(h5node, h5py.Dataset):
        varlist.append(vname)
    return 

with h5py.File(fn,'r') as h5pt:
    h5pt.visititems(IS2h5walk)
    
for tvar in varlist:
    vpath,vn = os.path.split(tvar)
    if vn==varname: print(tvar) 

#### Compare the varaible ```latitude``` in the original data and the subsetted dat

In [ ]:
region_a.variables['latitude']

In [ ]:
', '.join(x) for x in ['gt1l','gt1r']

## Look at variables from various datasets to generalize code

In [ ]:
region_06 = ipd.Icesat2Data('ATL06',[-55, 68, -48, 71],['2019-02-22','2019-02-28'], \
                           start_time='00:00:00', end_time='23:59:59', version='2')

In [ ]:
region_07 = ipd.Icesat2Data('ATL07',[-55, 68, -48, 71],['2019-02-22','2019-02-28'], \
                           start_time='00:00:00', end_time='23:59:59', version='2')

In [ ]:
region_08 = ipd.Icesat2Data('ATL08',[-55, 68, -48, 71],['2019-02-22','2019-02-28'], \
                           start_time='00:00:00', end_time='23:59:59', version='2')

In [ ]:
region_09 = ipd.Icesat2Data('ATL09',[-55, 68, -48, 71],['2019-02-22','2019-02-28'], \
                           start_time='00:00:00', end_time='23:59:59', version='2')

In [ ]:
region_10 = ipd.Icesat2Data('ATL10',[-55, 68, -48, 71],['2019-02-22','2019-02-28'], \
                           start_time='00:00:00', end_time='23:59:59', version='2')

In [ ]:
region_12 = ipd.Icesat2Data('ATL12',[-55, 68, -48, 71],['2019-02-22','2019-02-28'], \
                           start_time='00:00:00', end_time='23:59:59', version='2')

In [ ]:
session=region_a.earthdata_login('liuzheng','liuzheng@apl.uw.edu')

In [ ]:
session=region_06.earthdata_login('jessica.scheick','jessica.scheick@maine.edu')

In [ ]:
dset = region_10
dset.show_custom_options(session,dictview=True)

In [ ]:
## show the maximum depth for variables in dataset
for dset in [region_06, region_07, region_08, region_09, region_10, region_12]:
    dset._get_custom_options(session)
    max_dep = 0
    for vn in dset._cust_options['variables']:
        wrds = vn.split('/')
        if len(wrds)-1> max_dep: max_dep = len(wrds)-1
    print(dset.dataset,max_dep)

In [ ]:
vgrp, paths = region_10._parse_var_list(region_10._cust_options['variables'])
import pprint
pprint.pprint(vgrp)

In [ ]:
vgrp.keys()

In [ ]:
for dset in [region_06, region_07, region_08, region_09, region_10, region_12]:
    dset.show_custom_options(session, dictview=True)

In [ ]:
d=6
for dset in [region_06, region_07, region_08, region_09, region_10, region_12]:
    vgrp, paths = dset._parse_var_list(dset._cust_options['variables'])
    print(d)
    d=d+1
    for p in paths:
        print(np.unique(np.array(p)))
    print(np.unique(np.array(vgrp.keys())))